# AirBNB tracker

In [1]:
import datetime
import pandas as pd
import pyperclip
import re

In [2]:
temp = pyperclip.paste()

In [3]:
def tronque_à_gauche(texte: str) -> str:
    """Tronque à gauche le texte avant 'Classement des résultats'"""
    stop = texte.find('Classement des résultats')
    return texte[stop + len('Classement des résultats'):].strip()

def tronque_à_droite(texte: str) -> str:
    """Tronque à droite le texte qui dépasse de la numérotation des pages"""
    stop = texte.find('1\n2\n3\n4')
    return texte[:stop].strip()

In [4]:
temp = tronque_à_gauche(temp)
temp = tronque_à_droite(temp)

In [141]:
class Logement:
    __slots__ = [
        'date_enregistrement',
        'texte',
        'type',
        'ville',
        'description',
        'prix',
        'note',
        'nb_avis',
        'nb_lits',
        'nb_chambres',
        'est_professionnel',
        'est_nouveau'
    ]
    def __init__(self, ville: str, type: str, description: str,
                 nb_lits: int, nb_chambres: int,
                 est_professionnel: bool, est_nouveau: bool,
                 prix: int, note: float, nb_avis: int,
                 texte: str):
        self.type = type
        self.ville = ville
        self.description = description
        self.nb_lits = nb_lits
        self.nb_chambres = nb_chambres
        self.prix = prix
        self.est_nouveau = est_nouveau
        self.est_professionnel = est_professionnel
        self.note = note
        self.nb_avis = nb_avis

        self.date_enregistrement = datetime.date.today()
        self.texte = texte
    
    @classmethod
    def from_text(cls, texte):
        texte = texte.strip()
        #additional_attrs = datetime.date.today()
        attrs = set(cls.__slots__) - {'texte', 'date_enregistrement'}
        kwargs = dict(texte=texte)
        for attr in attrs:
            kwargs[attr] = eval(f'cls._get_{attr}')(texte)
        return cls(**kwargs)
    
    
    @staticmethod
    def _get_type(texte: str):
        return texte.split('\n')[0].split(' ⋅ ')[0]
    
    @staticmethod
    def _get_ville(texte: str):
        return texte.split('\n')[0].split(' ⋅ ')[1]

    @staticmethod
    def _get_description(text: str):
        return text.split('\n')[1]
    
    @staticmethod
    def _get_nb_lits(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[0]

    @staticmethod
    def _get_nb_chambres(texte: str):
        return Logement._get_nb_lits_nb_chambres(texte)[1]
    
    @staticmethod
    def _get_nb_lits_nb_chambres(texte: str) -> tuple:
        nb_lits, nb_chambres = None, None
        pat = r'(?P<nb_lits>\d) lits?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_lits = int(rgx.group()[0])
        pat = r'(?P<nb_chambres>\d) chambres?'
        if (rgx := re.search(pat, texte.split('\n')[2])):
            nb_chambres = int(rgx.group()[0])
        return nb_lits, nb_chambres
    
    @staticmethod
    def _get_est_professionnel(texte: str):
        return texte.split('\n')[3] == 'Professionnel'
    
    @staticmethod
    def _get_prix(texte: str):
        for ligne in texte.split('\n'):
            if rgx := re.match('(?P<prix>[\d\u202f]+) € par nuit', ligne):
                return int(rgx.groups()[0].replace('\u202f', ''))
        raise ValueError('Pas de prix trouvé sur :\n' + texte)
    
    @staticmethod
    def _get_est_nouveau(texte: str):
        return texte.split('\n')[-1] == 'Nouveau'
    
    @staticmethod
    def _get_note(texte: str):
        return Logement._get_note_et_nb_avis(texte)[0]

    @staticmethod
    def _get_nb_avis(texte: str):
        return Logement._get_note_et_nb_avis(texte)[1]

    @staticmethod
    def _get_note_et_nb_avis(texte: str):
        note, nb_avis = None, None
        pat = r'(?P<note>\d\,\d\d?) \((?P<nb_avis>\d+)\)'
        if rgx := re.search(pat, texte):
            note = float(rgx.groupdict()['note'].replace(',', '.'))
            nb_avis = int(rgx.groupdict()['nb_avis'])
        return note, nb_avis
    
    def __repr__(self):
        rep = f'{self.__class__.__name__}('
        for attr in ('type', 'ville', 'prix'):
            rep += f'\n    {attr}={getattr(self, attr)},'
        rep = rep[:-1] + '\n)'
        return rep

In [142]:
splits = [match.start() for match in re.finditer('(Appartement|Hébergement|Tiny house) ⋅ ', temp)]
splits += [len(temp)]
splits

[0,
 222,
 485,
 675,
 913,
 1179,
 1374,
 1580,
 1798,
 2022,
 2250,
 2449,
 2669,
 2867,
 3109,
 3329,
 3746,
 3980]

In [143]:
logements = []
for texte in [temp[start:stop] for start, stop in zip(splits, splits[1:])]:
    logements.append(Logement.from_text(texte))

In [144]:
dico = [{attr: getattr(logement, attr) for attr in Logement.__slots__} for logement in logements]
pd.DataFrame(dico, index=range(len(dico)))    

,date_enregistrement,texte,type,ville,description,prix,note,nb_avis,nb_lits,nb_chambres,est_professionnel,est_nouveau
0,2023-12-20,Appartement ⋅ Cachan\nAppt 2 chambres proche P...,Appartement,Cachan,Appt 2 chambres proche Paris!,278,NaN,NaN,NaN,NaN,False,True
1,2023-12-20,Hébergement ⋅ Cachan\nCharming Family French H...,Hébergement,Cachan,Charming Family French Home with Private Garden!,1290,4.85,13.0,NaN,NaN,False,False
2,2023-12-20,Appartement ⋅ Cachan\nT3 Cosy N°5 proche Paris...,Appartement,Cachan,T3 Cosy N°5 proche Paris,851,NaN,NaN,NaN,NaN,True,True
3,2023-12-20,Appartement ⋅ Cachan\nAppartement proche Paris...,Appartement,Cachan,"Appartement proche Paris, commerces et transports",315,NaN,NaN,NaN,NaN,False,False
4,2023-12-20,Hébergement ⋅ Cachan\nPetite maison tout confo...,Hébergement,Cachan,Petite maison tout confort parking terrasse ja...,280,5.00,7.0,NaN,NaN,False,False
5,2023-12-20,Appartement ⋅ Arcueil\nHome sweet home in Pari...,Appartement,Arcueil,Home sweet home in Paris,579,4.93,70.0,NaN,NaN,True,False
6,2023-12-20,Appartement ⋅ Cachan\nAppartement Entier Lumin...,Appartement,Cachan,Appartement Entier Lumineux Cachan 54m2,269,4.63,8.0,NaN,NaN,False,False
7,2023-12-20,Appartement ⋅ Bagneux\nBel appartement lumineu...,Appartement,Bagneux,Bel appartement lumineux,224,NaN,NaN,NaN,NaN,False,True
8,2023-12-20,Appartement ⋅ Cachan\nJoli 3 pièces à 10 min d...,Appartement,Cachan,Joli 3 pièces à 10 min de Paris,391,NaN,NaN,NaN,NaN,False,True
9,2023-12-20,Appartement ⋅ Cachan\nAppartement lumineux à 5...,Appartement,Cachan,Appartement lumineux à 5 min à pied du RER B,389,NaN,NaN,NaN,NaN,False,False
